In [1]:
require 'nn'
require 'hdf5'

In [2]:
local Squeeze, parent = torch.class('nn.Squeeze', 'nn.Module')

function Squeeze:updateOutput(input)
    self.size = input:size()
    self.output = input:squeeze()
    return self.output
end

function Squeeze:updateGradInput(input, gradOutput)
  self.gradInput = gradOutput:view(self.size)
  return self.gradInput  
end

In [3]:
myFile = hdf5.open('PTB.hdf5','r')
data = myFile:all()
myFile:close()

In [4]:
train_input_word_windows = data['train_input_word_windows']
train_input_cap_windows = data['train_input_cap_windows']

train = train_input_word_windows:clone()
train_cap = train_input_cap_windows:clone()
train_output = data['train_output']

In [10]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet = nn.Sequential()

--Include the lookup tables
neuralnet:add(nn.LookupTable(data['nwords'][1],dim_hidden))
neuralnet:add(nn.View(1,-1,5*50))
neuralnet:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet:add(nn.Linear(5*dim_hidden,dim_hidden2))
neuralnet:add(nn.HardTanh())
neuralnet:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet:add(nn.LogSoftMax())

In [6]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [7]:
criterion = nn.ClassNLLCriterion()

In [11]:
trainer = nn.StochasticGradient(neuralnet, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 10
trainer:train(dataset)

# StochasticGradient: training	


# current error = 0.76410843150984	


# current error = 0.49077146420375	


# current error = 0.39363261704525	


# current error = 0.33967493665589	


...

# current error = 0.2785490815371	


# current error = 0.25929846572195	


# current error = 0.24431658769046	


# current error = 0.23182629793747	


# current error = 0.22073000924219	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.22073000924219	


In [35]:
val_word = data['valid_input_word_windows']:clone()
val_output = data['valid_output']:clone()

pred_train = neuralnet:forward(train)
max,argmax_train = pred_train:max(2)
acc_train = 0
for i = 1, train_output:size(1) do
    if argmax_train[i][1] == train_output[i] then
        acc_train = acc_train + 1
    end
end
score_train = acc_train/train_output:size(1)
print('Validation Score on Train is '..score_train)

Validation Score on Train is 0.93472968205236	


In [36]:
pred_val = neuralnet:forward(val_word)
max,argmax_val = pred_val:max(2)
acc_val = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val[i][1] == data['valid_output'][i] then
        acc_val = acc_val + 1
    end
end
score_val = acc_val/data['valid_output']:size(1)
print('Validation Score on Train is '..score_val)

Validation Score on Train is 0.91536173828599	


## Adding the cap:

In [18]:
train_new = train_input_word_windows:clone()
train_cap_new = train_cap:clone()
train_cap_new:add(100002)

In [19]:
train_new = torch.cat(train_new,train_cap_new,2)

In [27]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet_wc = nn.Sequential()

--Include the lookup tables
neuralnet_wc:add(nn.LookupTable(data['nwords'][1]+4,dim_hidden))
neuralnet_wc:add(nn.View(1,-1,10*dim_hidden))
neuralnet_wc:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet_wc:add(nn.Linear(10*dim_hidden,dim_hidden2))
neuralnet_wc:add(nn.HardTanh())
neuralnet_wc:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet_wc:add(nn.LogSoftMax())

In [25]:
dataset2={};
for i=1,train:size(1) do 
  dataset2[i] = {train_new[i]:view(1,10), train_output[i]}
end
function dataset2:size() return train:size(1) end -- 100 examples

In [28]:
trainer2 = nn.StochasticGradient(neuralnet_wc, criterion)
trainer2.learningRate = 0.01
trainer2.maxIteration = 3
trainer2:train(dataset2)

# StochasticGradient: training	


# current error = 0.65287098094481	


# current error = 0.43838076996036	


# current error = 0.36003709605467	
# StochasticGradient: you have reached the maximum number of iterations	


# training error = 0.36003709605467	


In [29]:
pred_train2 = neuralnet_wc:forward(train_new)
max,argmax_train2 = pred_train2:max(2)
acc_train2 = 0
for i = 1, train_output:size(1) do
    if argmax_train2[i][1] == train_output[i] then
        acc_train2 = acc_train2 + 1
    end
end
score_train2 = acc_train2/train_output:size(1)
print('Validation Score on Train is '..score_train2)


Validation Score on Train is 0.90422454654824	


In [30]:
valid_new = data['valid_input_word_windows']:clone()
valid_cap_new = data['valid_input_cap_windows']:clone()
valid_cap_new:add(100002)
valid_new = torch.cat(valid_new,valid_cap_new,2)

In [32]:
pred_val2 = neuralnet_wc:forward(valid_new)
max,argmax_val2 = pred_val2:max(2)
acc_val2 = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val2[i][1] == data['valid_output'][i] then
        acc_val2 = acc_val2 + 1
    end
end
score_val2 = acc_val2/data['valid_output']:size(1)
print('Validation Score on Train is '..score_val2)

Validation Score on Train is 0.89842801650886	
